In [58]:
import numpy as np
import urllib
import os
import logging
from pathlib import Path
import pandas as pd
from sklearn.linear_model import LinearRegression

In [22]:
# Load in csvs
WHR_file_name = 'world-happiness-report.csv'
WHR2021_file_name = 'world-happiness-report-2021.csv'
CPDS_file_name = 'cleaned_CPDS.xlsx'
WHR_df = pd.read_csv(f'data\{WHR_file_name}')
WHR2021_df = pd.read_csv(f'data\{WHR2021_file_name}')
CPDS_df = pd.read_excel(f'data\{CPDS_file_name}')


# Clean Data
WHR_df = WHR_df.drop(columns = ['Positive affect','Negative affect'])
WHR2021_df['year'] = 2021
WHR2021_df = WHR2021_df.drop(columns = ['Standard error of ladder score','upperwhisker','lowerwhisker','Ladder score in Dystopia',
                          'Explained by: Log GDP per capita','Explained by: Social support','Explained by: Healthy life expectancy',
                          'Explained by: Freedom to make life choices','Explained by: Generosity','Explained by: Perceptions of corruption',
                          'Dystopia + residual'])
WHR2021_df = WHR2021_df.rename(columns = {'Ladder score':'Life Ladder','Logged GDP per capita':'Log GDP per capita',
                            'Healthy life expectancy':'Healthy life expectancy at birth'})

dic = {}
for i in WHR2021_df['Regional indicator'].unique():
    dic[i]=list(WHR2021_df[WHR2021_df['Regional indicator'] ==i].groupby('Country name').size().index)
dic['Sub-Saharan Africa'].append('Angola')
dic['Latin America and Caribbean'].append('Belize')
dic['South Asia'].append('Bhutan')
dic['Sub-Saharan Africa'].append('Central African Republic')
dic['Sub-Saharan Africa'].append('Congo (Kinshasa)')
dic['Latin America and Caribbean'].append('Cuba')
dic['Sub-Saharan Africa'].append('Djibouti')
dic['Latin America and Caribbean'].append('Guyana')
dic['Middle East and North Africa'].append('Oman')
dic['Middle East and North Africa'].append('Qatar')
dic['Sub-Saharan Africa'].append('Somalia')
dic['Sub-Saharan Africa'].append('Somaliland region')
dic['Sub-Saharan Africa'].append('South Sudan')
dic['Middle East and North Africa'].append('Sudan')
dic['Latin America and Caribbean'].append('Suriname')
dic['Middle East and North Africa'].append('Syria')
dic['Latin America and Caribbean'].append('Trinidad and Tobago')

def find_region(x):
    '''Helper Function to Return Region Name based on the Country Name entered'''
    for reg in dic.keys():
        for c in dic[reg]:
            if x == c:
                return reg
WHR_df['Regional indicator'] = WHR_df['Country name'].apply(find_region)

cmbd_WHR_df = pd.concat([WHR_df,WHR2021_df])


CPDS_df = CPDS_df.dropna(subset = ['year','country'])
def convert_to_int(x):
    return int(x)

CPDS_df['year'] = CPDS_df[['year']].applymap(convert_to_int)

all_data = pd.merge(cmbd_WHR_df,CPDS_df,how = 'left',left_on = ['year','Country name'],right_on = ['year','country'])
all_data = all_data.drop(columns = ['country'])
all_data

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Regional indicator,...,gov_type,vturn,womenpar,rae_ele,rae_leg,effpar_ele,effpar_leg,dis_abso,dis_rel,dis_gall
0,Afghanistan,2008,3.724,7.370,0.451,50.800,0.718,0.168,0.882,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2009,4.402,7.540,0.552,51.200,0.679,0.190,0.850,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2010,4.758,7.647,0.539,51.600,0.600,0.121,0.707,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2011,3.832,7.620,0.521,51.920,0.496,0.162,0.731,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2012,3.783,7.705,0.521,52.240,0.531,0.236,0.776,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,Lesotho,2021,3.512,7.926,0.787,48.700,0.715,-0.131,0.915,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,Botswana,2021,3.467,9.782,0.784,59.269,0.824,-0.246,0.801,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095,Rwanda,2021,3.415,7.676,0.552,61.400,0.897,0.061,0.167,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2096,Zimbabwe,2021,3.145,7.943,0.750,56.201,0.677,-0.047,0.821,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
cmbd_WHR_df[(cmbd_WHR_df['year'] >= 2006) & (cmbd_WHR_df['year'] <= 2021)].isnull().sum()

Country name                          0
year                                  0
Life Ladder                           0
Log GDP per capita                   36
Social support                       13
Healthy life expectancy at birth     55
Freedom to make life choices         30
Generosity                           63
Perceptions of corruption           108
Regional indicator                    0
dtype: int64

In [24]:
cmbd_WHR_df.isnull().sum()

Country name                          0
year                                  0
Life Ladder                           0
Log GDP per capita                   36
Social support                       13
Healthy life expectancy at birth     55
Freedom to make life choices         32
Generosity                           89
Perceptions of corruption           110
Regional indicator                    0
dtype: int64

In [25]:
all_data.columns

Index(['Country name', 'year', 'Life Ladder', 'Log GDP per capita',
       'Social support', 'Healthy life expectancy at birth',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Regional indicator', 'poco', 'eu', 'emu',
       'gov_right1', 'gov_cent1', 'gov_left1', 'gov_new', 'gov_gap',
       'gov_chan', 'gov_right2', 'gov_cent2', 'gov_left2', 'gov_right3',
       'gov_cent3', 'gov_left3', 'gov_sup', 'gov_type', 'vturn', 'womenpar',
       'rae_ele', 'rae_leg', 'effpar_ele', 'effpar_leg', 'dis_abso', 'dis_rel',
       'dis_gall'],
      dtype='object')

In [40]:
years_present_df = cmbd_WHR_df[(cmbd_WHR_df['year'] >= 2006) & (cmbd_WHR_df['year'] <= 2021)].groupby('Country name').count()[['year']]
countries_to_include = list(years_present_df[years_present_df['year'] > 8].index)

In [6]:
temp_df = cmbd_WHR_df.drop(columns = ['Regional indicator'])

In [57]:

years_present_df = cmbd_WHR_df[(cmbd_WHR_df['year'] >= 2006) & (cmbd_WHR_df['year'] <= 2021)].groupby('Country name').count()[['year']]
countries_to_include = list(years_present_df[years_present_df['year'] > 10].index)
included_years = [i for i in range(2007,2022)]
columns = temp_df.columns[2:]
new_col_names = []
for i in included_years:
    year = str(i)
    for col in columns:
        col_name = f'{col}_{year}'
        new_col_names.append(col_name)

groups = temp_df.groupby('Country name')

num_cols = len(columns)
missing_year_data = [np.nan for i in range(num_cols)]
data = {}
for key, group in groups:
    if key in countries_to_include:
        row_data = []
        group_years = list(group['year'])
        c_group = group.drop(columns = 'Country name')
        c_group = c_group.set_index('year')
        for i in included_years:
            if i in group_years:
                row_data += list(c_group.loc[i])
            else:
                row_data += missing_year_data
                
        data[key] = row_data
    

reformatted_data = pd.DataFrame.from_dict(data, orient='index',columns = new_col_names)

ttl_nulls = pd.DataFrame(reformatted_data.isnull().sum()).sum()
ttl_vals = reformatted_data.shape[0] * reformatted_data.shape[1]
pct_missing = ttl_nulls / ttl_vals
print(pct_missing)

reformatted_data

0    0.095688
dtype: float64


,Life Ladder_2007,Log GDP per capita_2007,Social support_2007,Healthy life expectancy at birth_2007,Freedom to make life choices_2007,Generosity_2007,Perceptions of corruption_2007,Life Ladder_2008,Log GDP per capita_2008,Social support_2008,...,Freedom to make life choices_2020,Generosity_2020,Perceptions of corruption_2020,Life Ladder_2021,Log GDP per capita_2021,Social support_2021,Healthy life expectancy at birth_2021,Freedom to make life choices_2021,Generosity_2021,Perceptions of corruption_2021
Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.724,7.370,0.451,...,NaN,NaN,NaN,2.523,7.695,0.463,52.493,0.382,-0.102,0.924
Albania,4.634,9.142,0.821,65.80,0.529,-0.009,0.875,NaN,NaN,NaN,...,0.754,0.007,0.891,5.117,9.520,0.697,68.999,0.785,-0.030,0.901
Argentina,6.073,10.018,0.862,66.94,0.653,-0.141,0.881,5.961,10.048,0.892,...,0.823,-0.122,0.816,5.929,9.962,0.898,69.000,0.828,-0.182,0.834
Armenia,4.882,9.181,0.760,64.90,0.605,-0.251,0.817,4.652,9.256,0.709,...,NaN,NaN,NaN,5.283,9.487,0.799,67.055,0.825,-0.168,0.629
Australia,7.285,10.703,0.965,71.72,0.891,0.347,0.513,7.254,10.719,0.947,...,0.905,0.210,0.491,7.183,10.796,0.940,73.900,0.914,0.159,0.442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.258,9.701,0.922,...,0.612,NaN,0.811,4.892,9.073,0.861,66.700,0.615,-0.169,0.827
Vietnam,5.422,8.394,0.856,66.02,0.918,0.089,0.754,5.480,8.440,0.805,...,NaN,NaN,NaN,5.411,8.973,0.850,68.034,0.940,-0.098,0.796
Yemen,4.477,8.214,0.825,53.40,0.673,0.011,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.658,7.578,0.832,57.122,0.602,-0.147,0.800
Zambia,3.998,7.871,0.688,45.72,0.682,-0.067,0.948,4.730,7.918,0.624,...,0.750,0.056,0.810,4.073,8.145,0.708,55.809,0.782,0.061,0.823


In [75]:
years_present_df = cmbd_WHR_df[(cmbd_WHR_df['year'] >= 2006) & (cmbd_WHR_df['year'] <= 2021)].groupby('Country name').count()[['year']]
countries_to_include = list(years_present_df[years_present_df['year'] > 10].index)
included_years = [i for i in range(2007,2022)]
columns = temp_df.columns[2:]
new_col_names = []
for col in columns:
    for i in included_years:
        year = str(i)
        col_name = f'{col}_{year}'
        new_col_names.append(col_name)

groups = temp_df.groupby('Country name')

num_cols = len(columns)
missing_year_data = [np.nan for i in range(num_cols)]
data = {}
for key, group in groups:
    if key in countries_to_include:
        row_data = []
        group_years = list(group['year'])
        c_group = group.drop(columns = 'Country name')
        c_group = c_group.set_index('year')
        for col in columns:
            col_years = []
            col_vals = []
            for year in range(2007,2022):
                try:
                    t = c_group.loc[year,col]
                    if t == t:
                        col_years.append(year)
                        col_vals.append(t)
                except:
                    t = 0  
            X = pd.DataFrame({'year':col_years})
            Y = pd.DataFrame({col:col_vals})[col]
            col_predictor = LinearRegression().fit(X,Y)
            for year in range(2007,2022):
                try:
                    t = c_group.loc[year,col]
                    if t == t:
                        row_data.append(t)
                    else:
                        predicted_value = col_predictor.predict(pd.DataFrame({'year':[year]}))[0]    
                        row_data.append(predicted_value) 
                except:
                    predicted_value = col_predictor.predict(pd.DataFrame({'year':[year]}))[0]    
                    row_data.append(predicted_value) 
        data[key] = row_data
    

reformatted_data = pd.DataFrame.from_dict(data, orient='index',columns = new_col_names)

ttl_nulls = pd.DataFrame(reformatted_data.isnull().sum()).sum()
ttl_vals = reformatted_data.shape[0] * reformatted_data.shape[1]
pct_missing = ttl_nulls / ttl_vals
print(pct_missing)

reformatted_data

0    0.0
dtype: float64


,Life Ladder_2007,Life Ladder_2008,Life Ladder_2009,Life Ladder_2010,Life Ladder_2011,Life Ladder_2012,Life Ladder_2013,Life Ladder_2014,Life Ladder_2015,Life Ladder_2016,...,Perceptions of corruption_2012,Perceptions of corruption_2013,Perceptions of corruption_2014,Perceptions of corruption_2015,Perceptions of corruption_2016,Perceptions of corruption_2017,Perceptions of corruption_2018,Perceptions of corruption_2019,Perceptions of corruption_2020,Perceptions of corruption_2021
Afghanistan,4.558258,3.724000,4.402000,4.758000,3.832,3.783,3.572,3.131,3.983,4.220,...,0.776,0.823,0.871000,0.881000,0.793000,0.954000,0.928,0.924,0.920688,0.924
Albania,4.634000,5.125414,5.485000,5.269000,5.867,5.510,4.551,4.814,4.607,4.511,...,0.848,0.863,0.883000,0.885000,0.901000,0.876000,0.899,0.914,0.891000,0.901
Argentina,6.073000,5.961000,6.424000,6.441000,6.776,6.468,6.582,6.671,6.697,6.427,...,0.817,0.823,0.854000,0.851000,0.851000,0.841000,0.855,0.830,0.816000,0.834
Armenia,4.882000,4.652000,4.178000,4.368000,4.260,4.320,4.277,4.453,4.348,4.325,...,0.893,0.900,0.920000,0.901000,0.921000,0.865000,0.677,0.583,0.726556,0.629
Australia,7.285000,7.254000,7.336073,7.450000,7.406,7.196,7.364,7.289,7.309,7.250,...,0.368,0.432,0.442000,0.357000,0.399000,0.411000,0.405,0.430,0.491000,0.442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,7.423220,6.258000,7.189000,7.478000,6.580,7.067,6.553,6.136,5.569,4.041,...,0.743,0.837,0.827000,0.813000,0.890000,0.844000,0.828,0.839,0.811000,0.827
Vietnam,5.422000,5.480000,5.304000,5.296000,5.767,5.535,5.023,5.085,5.076,5.062,...,0.815,0.771,0.787337,0.789108,0.799000,0.792650,0.808,0.788,0.797962,0.796
Yemen,4.477000,4.387173,4.809000,4.350000,3.746,4.061,4.218,3.968,2.983,3.826,...,0.793,0.885,0.885000,0.829000,0.817769,0.815308,0.793,0.798,0.807923,0.800
Zambia,3.998000,4.730000,5.260000,4.748332,4.999,5.013,5.244,4.346,4.843,4.348,...,0.806,0.732,0.809000,0.871000,0.771000,0.740000,0.811,0.832,0.810000,0.823


In [66]:
pd.DataFrame({'year':[1,2,3]})
pd.DataFrame({'vals':[1,2,3]})['vals']

0    1
1    2
2    3
Name: vals, dtype: int64

In [60]:
included_years = [i for i in range(2007,2022)]
included_years

[2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021]

In [76]:
reformatted_data.to_csv('final_data/WHD_with_no_missing_data.csv')

In [55]:
pd.DataFrame(reformatted_data.isnull().sum()).sort_values(by  = 0, ascending = False).head(20)

,0
Generosity_2020,46
Log GDP per capita_2020,46
Perceptions of corruption_2020,45
Healthy life expectancy at birth_2020,42
Freedom to make life choices_2020,40
Social support_2020,39
Life Ladder_2020,39
Perceptions of corruption_2007,36
Social support_2007,34
Healthy life expectancy at birth_2007,33


In [56]:
ttl_nulls = pd.DataFrame(reformatted_data.isnull().sum()).sum()
ttl_vals = reformatted_data.shape[0] * reformatted_data.shape[1]
pct_missing = ttl_nulls / ttl_vals
pct_missing

0    0.095688
dtype: float64

In [43]:
def convert_to_int(x):
    return int(x)

CPDS_df['year'] = CPDS_df[['year']].applymap(convert_to_int)
CPDS_df.head()

,year,country,countryn,iso,iso3n,cpds1,poco,eu,emu,gov_right1,...,emprot_temp,prefisc_gini,pretran_gini,postfisc_gini,pop,pop15_64,pop65,elderly,year_13,country_13
0,1960,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10249.099609,6353.600098,849.000000,8.283654,1960.0,Australia
1,1961,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10452.099609,6486.899902,868.099976,8.305508,1961.0,Australia
2,1962,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10649.500000,6631.100098,886.500000,8.324334,1962.0,Australia
3,1963,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10851.200195,6771.700195,905.400024,8.343778,1963.0,Australia
4,1964,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,11071.299805,6919.500000,920.000000,8.309773,1964.0,Australia
